# 数据预处理

In [1]:
import pandas as pd

In [2]:
df_MDtr = pd.read_excel("Molecular_Descriptor.xlsx",sheet_name = "training")
df_MDte = pd.read_excel("Molecular_Descriptor.xlsx",sheet_name = "test")
df_ER = pd.read_excel("ERα_activity.xlsx")

# 将标准差为0的删掉

In [3]:
df_MDtr = df_MDtr.loc[:,df_MDtr.describe().T[df_MDtr.describe().T["std"] != 0].index]

# 孤立森林处理异常

In [4]:
import plotly.express as px
from sklearn.ensemble import IsolationForest

iforest = IsolationForest(n_estimators=100, max_samples='auto', 
                          contamination=0.05, max_features=73, 
                          bootstrap=False, n_jobs=-1, random_state=1)

x=df_MDtr.iloc[:,1:]
pred= iforest.fit_predict(x)
df_MDtr['scores']=iforest.decision_function(x)
df_MDtr['anomaly_label']=pred
data_yichang=df_MDtr[df_MDtr.anomaly_label==-1]
data_yichang.to_csv("isoForest.csv")

df_MD_nor = df_MDtr[df_MDtr.anomaly_label==1]

# 数据的拼接

In [7]:
lst_nor = list(df_MD_nor.index)
df_MD_nor = df_MD_nor.iloc[:,:-2]

In [8]:
adm = pd.read_excel("ADMET.xlsx")
df_MD_nor["MN"] = adm.loc[lst_nor,"MN"]

In [10]:
df_MD_nor = pd.merge(df_ER.loc[lst_nor,"pIC50"], df_MD_nor, left_index = True, right_index = True)

In [12]:
df_MD_nor = df_MD_nor.reset_index()
df_MD_nor = df_MD_nor.drop("index",axis=1)

# MIC

In [10]:
y = df_MD_nor['pIC50']
X = df_MD_nor.drop('pIC50', axis=1)
X_train ,X_test ,y_train, y_test = train_test_split(X,y,test_size=0.3)

<IPython.core.display.Javascript object>

In [14]:
all_ranking = pd.read_csv("all_features_ranking.csv")
fea_can_lst = list(all_ranking.loc[all_ranking["avg_ranking"]<=35].key_0)
fea_can_lst.append("pIC50")
fea_can_35 = df_MD_nor.loc[:,fea_can_lst]
# fea_can_35.to_csv("fea_can_35.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'all_features_ranking.csv'

In [11]:
from minepy import MINE
import operator

mine = MINE()
mic_scores = []
for col in X_train.columns:
    mine.compute_score(X_train.loc[:,col],y_train)
    mic_scores.append(mine.mic())
mic_ranking = sorted(list(zip(X_train.columns, mic_scores)), key=operator.itemgetter(1), reverse =True)
fea_name = []
fea_score = []
for i in range(len(mic_ranking)):
    fea_name.append(mic_ranking[i][0])
    fea_score.append(mic_ranking[i][1])
mic_df = pd.DataFrame(fea_score,index=fea_name)

# Catboost 

In [14]:
from catboost import CatBoostRegressor
cbc = CatBoostRegressor()
cbc.fit(X_train,y_train)
cbc_importance = cbc.feature_importances_
indices = np.argsort(cbc_importance)[::-1]
features = X.columns
cbc_feature_list = []
cbc_importance_list = []
for f in range(X.shape[1]):
    print("%2d %-*s %f" % (f + 1, 30, features[indices[f]], cbc_importance[indices[f]]))
    cbc_feature_list.append(features[indices[f]])
    cbc_importance_list.append(cbc_importance[indices[f]])
cbc_df = pd.DataFrame(cbc_importance_list,cbc_feature_list)
cbc.score(X_test,y_test)

Learning rate set to 0.040963
0:	learn: 0.3998938	total: 181ms	remaining: 3m
1:	learn: 0.3909295	total: 206ms	remaining: 1m 42s
2:	learn: 0.3834675	total: 230ms	remaining: 1m 16s
3:	learn: 0.3776006	total: 255ms	remaining: 1m 3s
4:	learn: 0.3726967	total: 277ms	remaining: 55.2s
5:	learn: 0.3658765	total: 309ms	remaining: 51.2s
6:	learn: 0.3590616	total: 330ms	remaining: 46.8s
7:	learn: 0.3531097	total: 351ms	remaining: 43.6s
8:	learn: 0.3464429	total: 373ms	remaining: 41.1s
9:	learn: 0.3408885	total: 395ms	remaining: 39.1s
10:	learn: 0.3354893	total: 419ms	remaining: 37.7s
11:	learn: 0.3306699	total: 443ms	remaining: 36.5s
12:	learn: 0.3258743	total: 465ms	remaining: 35.3s
13:	learn: 0.3221686	total: 488ms	remaining: 34.4s
14:	learn: 0.3177073	total: 509ms	remaining: 33.5s
15:	learn: 0.3138118	total: 531ms	remaining: 32.7s
16:	learn: 0.3099566	total: 554ms	remaining: 32s
17:	learn: 0.3055907	total: 575ms	remaining: 31.4s
18:	learn: 0.3004824	total: 598ms	remaining: 30.9s
19:	learn: 0.2

170:	learn: 0.1464284	total: 3.96s	remaining: 19.2s
171:	learn: 0.1459859	total: 3.98s	remaining: 19.2s
172:	learn: 0.1459066	total: 4.01s	remaining: 19.2s
173:	learn: 0.1456870	total: 4.03s	remaining: 19.1s
174:	learn: 0.1454722	total: 4.05s	remaining: 19.1s
175:	learn: 0.1453248	total: 4.08s	remaining: 19.1s
176:	learn: 0.1450876	total: 4.1s	remaining: 19.1s
177:	learn: 0.1445072	total: 4.12s	remaining: 19s
178:	learn: 0.1440438	total: 4.14s	remaining: 19s
179:	learn: 0.1437176	total: 4.17s	remaining: 19s
180:	learn: 0.1434858	total: 4.19s	remaining: 19s
181:	learn: 0.1433434	total: 4.21s	remaining: 18.9s
182:	learn: 0.1429385	total: 4.23s	remaining: 18.9s
183:	learn: 0.1426246	total: 4.25s	remaining: 18.9s
184:	learn: 0.1424045	total: 4.28s	remaining: 18.8s
185:	learn: 0.1420404	total: 4.3s	remaining: 18.8s
186:	learn: 0.1414944	total: 4.32s	remaining: 18.8s
187:	learn: 0.1414501	total: 4.34s	remaining: 18.8s
188:	learn: 0.1410698	total: 4.37s	remaining: 18.7s
189:	learn: 0.1406369	

338:	learn: 0.1045150	total: 7.78s	remaining: 15.2s
339:	learn: 0.1043500	total: 7.8s	remaining: 15.1s
340:	learn: 0.1042812	total: 7.83s	remaining: 15.1s
341:	learn: 0.1037617	total: 7.85s	remaining: 15.1s
342:	learn: 0.1035243	total: 7.87s	remaining: 15.1s
343:	learn: 0.1032741	total: 7.89s	remaining: 15.1s
344:	learn: 0.1032152	total: 7.92s	remaining: 15s
345:	learn: 0.1031245	total: 7.94s	remaining: 15s
346:	learn: 0.1030149	total: 7.96s	remaining: 15s
347:	learn: 0.1028195	total: 7.99s	remaining: 15s
348:	learn: 0.1026955	total: 8.01s	remaining: 14.9s
349:	learn: 0.1022705	total: 8.03s	remaining: 14.9s
350:	learn: 0.1022088	total: 8.06s	remaining: 14.9s
351:	learn: 0.1019426	total: 8.08s	remaining: 14.9s
352:	learn: 0.1019120	total: 8.1s	remaining: 14.8s
353:	learn: 0.1018515	total: 8.12s	remaining: 14.8s
354:	learn: 0.1018042	total: 8.15s	remaining: 14.8s
355:	learn: 0.1012902	total: 8.19s	remaining: 14.8s
356:	learn: 0.1007963	total: 8.21s	remaining: 14.8s
357:	learn: 0.1006322	

502:	learn: 0.0787087	total: 11.5s	remaining: 11.4s
503:	learn: 0.0785628	total: 11.6s	remaining: 11.4s
504:	learn: 0.0783736	total: 11.6s	remaining: 11.4s
505:	learn: 0.0783504	total: 11.6s	remaining: 11.3s
506:	learn: 0.0780100	total: 11.6s	remaining: 11.3s
507:	learn: 0.0777302	total: 11.7s	remaining: 11.3s
508:	learn: 0.0775565	total: 11.7s	remaining: 11.3s
509:	learn: 0.0773684	total: 11.7s	remaining: 11.2s
510:	learn: 0.0773270	total: 11.7s	remaining: 11.2s
511:	learn: 0.0770696	total: 11.8s	remaining: 11.2s
512:	learn: 0.0769916	total: 11.8s	remaining: 11.2s
513:	learn: 0.0767658	total: 11.8s	remaining: 11.2s
514:	learn: 0.0766546	total: 11.8s	remaining: 11.1s
515:	learn: 0.0765031	total: 11.8s	remaining: 11.1s
516:	learn: 0.0763309	total: 11.9s	remaining: 11.1s
517:	learn: 0.0761824	total: 11.9s	remaining: 11.1s
518:	learn: 0.0761146	total: 11.9s	remaining: 11s
519:	learn: 0.0759379	total: 11.9s	remaining: 11s
520:	learn: 0.0756963	total: 12s	remaining: 11s
521:	learn: 0.075590

665:	learn: 0.0603676	total: 15.3s	remaining: 7.66s
666:	learn: 0.0602813	total: 15.3s	remaining: 7.64s
667:	learn: 0.0601020	total: 15.3s	remaining: 7.62s
668:	learn: 0.0600464	total: 15.3s	remaining: 7.59s
669:	learn: 0.0598838	total: 15.4s	remaining: 7.57s
670:	learn: 0.0597969	total: 15.4s	remaining: 7.55s
671:	learn: 0.0595512	total: 15.4s	remaining: 7.52s
672:	learn: 0.0594174	total: 15.4s	remaining: 7.5s
673:	learn: 0.0594061	total: 15.5s	remaining: 7.48s
674:	learn: 0.0592676	total: 15.5s	remaining: 7.46s
675:	learn: 0.0591612	total: 15.5s	remaining: 7.43s
676:	learn: 0.0589573	total: 15.5s	remaining: 7.41s
677:	learn: 0.0587766	total: 15.6s	remaining: 7.39s
678:	learn: 0.0587523	total: 15.6s	remaining: 7.37s
679:	learn: 0.0587413	total: 15.6s	remaining: 7.34s
680:	learn: 0.0585545	total: 15.6s	remaining: 7.32s
681:	learn: 0.0584294	total: 15.7s	remaining: 7.3s
682:	learn: 0.0583512	total: 15.7s	remaining: 7.27s
683:	learn: 0.0583069	total: 15.7s	remaining: 7.25s
684:	learn: 0.

829:	learn: 0.0459609	total: 19s	remaining: 3.89s
830:	learn: 0.0459007	total: 19s	remaining: 3.87s
831:	learn: 0.0458667	total: 19.1s	remaining: 3.85s
832:	learn: 0.0457764	total: 19.1s	remaining: 3.83s
833:	learn: 0.0457652	total: 19.1s	remaining: 3.8s
834:	learn: 0.0456790	total: 19.1s	remaining: 3.78s
835:	learn: 0.0456155	total: 19.1s	remaining: 3.76s
836:	learn: 0.0455983	total: 19.2s	remaining: 3.73s
837:	learn: 0.0455333	total: 19.2s	remaining: 3.71s
838:	learn: 0.0455037	total: 19.2s	remaining: 3.69s
839:	learn: 0.0454077	total: 19.2s	remaining: 3.66s
840:	learn: 0.0453350	total: 19.3s	remaining: 3.64s
841:	learn: 0.0452824	total: 19.3s	remaining: 3.62s
842:	learn: 0.0451575	total: 19.3s	remaining: 3.6s
843:	learn: 0.0451254	total: 19.3s	remaining: 3.57s
844:	learn: 0.0450411	total: 19.4s	remaining: 3.55s
845:	learn: 0.0449874	total: 19.4s	remaining: 3.53s
846:	learn: 0.0449276	total: 19.4s	remaining: 3.5s
847:	learn: 0.0449103	total: 19.4s	remaining: 3.48s
848:	learn: 0.04478

988:	learn: 0.0358520	total: 22.6s	remaining: 251ms
989:	learn: 0.0357850	total: 22.6s	remaining: 228ms
990:	learn: 0.0356793	total: 22.6s	remaining: 205ms
991:	learn: 0.0356341	total: 22.6s	remaining: 183ms
992:	learn: 0.0355486	total: 22.7s	remaining: 160ms
993:	learn: 0.0355424	total: 22.7s	remaining: 137ms
994:	learn: 0.0355359	total: 22.7s	remaining: 114ms
995:	learn: 0.0355099	total: 22.7s	remaining: 91.3ms
996:	learn: 0.0354409	total: 22.8s	remaining: 68.5ms
997:	learn: 0.0353825	total: 22.8s	remaining: 45.7ms
998:	learn: 0.0353132	total: 22.8s	remaining: 22.8ms
999:	learn: 0.0353073	total: 22.8s	remaining: 0us
 1 WTPT-5                         12.888707
 2 ETA_BetaP_s                    5.262450
 3 nHBAcc_Lipinski                4.525832
 4 ETA_EtaP_B_RC                  4.270713
 5 TopoPSA                        3.678875
 6 nssCH2                         3.029319
 7 nN                             2.866446
 8 maxsssCH                       2.653068
 9 WTPT-3                    

# 随机森林

In [16]:
from sklearn.ensemble import RandomForestRegressor

rfc = RandomForestRegressor(n_estimators=100)
rfc.fit(X_train, y_train)
rfc_importance = rfc.feature_importances_
rfc_importance = rfc_importance * 100
indices = np.argsort(rfc_importance)[::-1]
features = X.columns
rfc_feature_list = []
rfc_importance_list = []
for f in range(X.shape[1]):
    print("%2d %-*s %f" % (f + 1, 30, features[indices[f]], rfc_importance[indices[f]]))
    rfc_feature_list.append(features[indices[f]])
    rfc_importance_list.append(rfc_importance[indices[f]])
rfc_df = pd.DataFrame(rfc_importance_list,rfc_feature_list)
rfc.score(X_test,y_test)

 1 WTPT-3                         16.570871
 2 ETA_EtaP_B_RC                  9.269142
 3 WTPT-5                         7.709687
 4 ETA_BetaP_s                    7.102090
 5 nHBAcc_Lipinski                6.654668
 6 FMF                            3.119224
 7 maxsssCH                       2.153917
 8 nHBAcc                         1.616594
 9 maxsCH3                        1.540843
10 ETA_Epsilon_1                  1.524256
11 mindssC                        1.192751
12 TopoPSA                        1.152062
13 LipoaffinityIndex              0.923994
14 MAXDN2                         0.741012
15 ALogp2                         0.722225
16 ETA_Epsilon_4                  0.670908
17 MAXDN                          0.648333
18 SssCH2                         0.645995
19 SsssN                          0.632584
20 ETA_dBeta                      0.629319
21 minsCH3                        0.628952
22 SsssCH                         0.549078
23 SHother                        0.547189
24 SaasC  

In [18]:
rfc_df["ranking"] = range(len(rfc_df))
mic_df["ranking"] = range(len(mic_df))
cbc_df["ranking"] = range(len(cbc_df))
rfc_df["ranking"] = rfc_df["ranking"]+1
mic_df["ranking"] = mic_df["ranking"]+1
cbc_df["ranking"] = cbc_df["ranking"]+1

In [19]:
two_df = pd.merge(mic_df,rfc_df,left_on=mic_df.index,right_on=rfc_df.index).set_index("key_0")
three_df = pd.merge(two_df,cbc_df,left_on=two_df.index,right_on=cbc_df.index).set_index("key_0")
three_df = three_df.rename(columns={"0_x":"mic_score","ranking_x":"mic_ranking","0_y":"rfc_score","ranking_y":"rfc_ranking",0:"cbc_score","ranking":"cbc_ranking"})
three_df["avg_ranking"] = (three_df.mic_ranking + three_df.rfc_ranking + three_df.cbc_ranking)/3
three_df = three_df.sort_values("avg_ranking")
three_df["avg_ranking"] = range(len(three_df))
three_df["avg_ranking"] = three_df["avg_ranking"] + 1
three_df.to_csv("caclog_features_ranking.csv")
# three_df.to_csv("all_feature_ranking.csv")

In [20]:
three_df.rename_axis("features",inplace=True)
feature_lst = list(three_df.loc[three_df["avg_ranking"]<=20].index)
feature_lst.append("MN")
model_data_ = df_MD_nor.loc[:,feature_lst]
model_data_.to_csv("MN_loge.csv")

In [106]:
admet = pd.read_excel("ADMET.xlsx")
admet = admet.loc[lst_nor,:].reset_index()
admet.drop("index", axis=1, inplace = True)
df_MD_nor.drop("pIC50", axis = 1, inplace = True )

In [75]:
cac = admet.loc[:,"Caco-2"]
cyp = admet.loc[:,"CYP3A4"]
her = admet.loc[:,"hERG"]
hob = admet.loc[:,"HOB"]
mn  = admet.loc[:,"MN"]
cac_df = pd.merge(df_MD_nor,cac,left_index = True, right_index = True)
cyp_df = pd.merge(df_MD_nor,cyp,left_index = True, right_index = True)
her_df = pd.merge(df_MD_nor,her,left_index = True, right_index = True)
hob_df = pd.merge(df_MD_nor,hob,left_index = True, right_index = True)
mn_df = pd.merge(df_MD_nor,mn,left_index = True, right_index = True)
cac_df.to_csv("cac_df.csv")
cyp_df.to_csv("cyp_df.csv")
her_df.to_csv("her_df.csv")
hob_df.to_csv("hob_df.csv")
mn_df.to_csv("mn_df.csv")